<a href="https://colab.research.google.com/github/BlazingWarlord/Mental-Health-Support-Tools/blob/main/Tag-based%20Mental%20Health%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow stopwords wordnet nltk

In [ ]:
import json
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

# Load the dataset
lemmatizer = WordNetLemmatizer()
data_file = 'intents.json'

with open(data_file) as file:
    data = json.load(file)

nltk.download('punkt')
nltk.download('wordnet')


# Preprocess the data
def preprocess_data():
    words = []
    classes = []
    documents = []
    ignore_words = ['?', '!']

    for intent in data['intents']:
        for pattern in intent['patterns']:
            # Tokenize each word
            word_list = nltk.word_tokenize(pattern)
            words.extend(word_list)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    # Lemmatize and lower each word and remove duplicates
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_words]
    words = sorted(list(set(words)))
    classes = sorted(list(set(classes)))

    return words, classes, documents

words, classes, documents = preprocess_data()


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def prepare_training_data(words, classes, documents):
    training = []
    output_empty = [0] * len(classes)

    for doc in documents:
        bag = []
        pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
        for word in words:
            bag.append(1 if word in pattern_words else 0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])

    # Print the lengths of bag and output_row for each document to check for inconsistencies
    for item in training:
        print(f"Length of bag: {len(item[0])}, Length of output_row: {len(item[1])}")

    training = np.array(training, dtype=object) # Use dtype=object to handle inconsistent shapes
    X = np.array([item[0] for item in training])
    y = np.array([item[1] for item in training])

    return X, y

X, y = prepare_training_data(words, classes, documents)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of bag: 297, Length of output_row: 80
Length of 

In [ ]:
def build_model(input_shape, output_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=(input_shape,), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model(input_shape=len(X[0]), output_shape=len(y[0]))

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0233 - loss: 4.3954
Epoch 2/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0229 - loss: 4.3279
Epoch 3/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0670 - loss: 4.3278
Epoch 4/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1252 - loss: 4.1825
Epoch 5/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1251 - loss: 4.0784
Epoch 6/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0826 - loss: 4.0135
Epoch 7/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1122 - loss: 3.8201
Epoch 8/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2140 - loss: 3.5646
Epoch 9/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1745 - loss: 3.5972
Epoch 10/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1776 - loss: 3.3026
Epoch 11/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2272 - loss: 3.1999
Epoch 12/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2517 - lo

In [ ]:
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model



# Function to clean and preprocess user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert user input into a bag of words
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

# Function to classify intent based on user input
def classify_intent(sentence):
    bow_vec = bow(sentence, words)
    res = model.predict(np.array([bow_vec]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

# Function to get the chatbot's response based on classified intent
def get_response(intents_list, intents):
    if intents_list:
        tag = intents_list[0][0]
        for intent in intents['intents']:
            if intent['tag'] == tag:
                return random.choice(intent['responses'])
    return "I didn't understand that."

# Infinite chatbot loop
while True:
    message = input("You: ")
    tag = classify_intent(message)
    print(tag)
    response = get_response(tag, data)
    print(f"Bot: {response}")